In [ ]:
# Импорт всех необходимых модулей для оптимизации
import sys
import os
from pathlib import Path

# Добавляем путь к модулям проекта
project_root = Path.cwd()
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

# Основные модули оптимизации
from optimization import (
    # Основные классы
    OptimizationExecutor,
    OptimizationReporter,
    
    # Хранилище результатов
    InMemoryResultsStore,
    OptimizationResult,
    ResultsStoreProtocol,
    
    # Пространство параметров
    ParameterDefinition,
    ParameterSpace,
    
    # Стратегии поиска
    SearchStrategy,
    GridSearchStrategy,
    RandomSearchStrategy,
)

# Дополнительные импорты для работы с данными и бэктестингом
import pandas as pd
import numpy as np
from typing import Dict, Any, List, Optional, Mapping, Sequence
from dataclasses import dataclass
from datetime import datetime, UTC

# Импорты для работы с сигналами и индикаторами
from signals.base_signal import BaseSignal
from signals.signal_registry import SignalRegistry
from indicators.base_indicator import BaseIndicator
from indicators.indicator_registry import IndicatorRegistry

# Импорты для бэктестинга
from backtest.engine import BacktestEngine
from backtest.metrics import MetricsCalculator
from backtest.trade import Trade
from backtest.commission import CommissionCalculator
from optimization.executor import SignalConfig

print("✅ Все модули оптимизации успешно импортированы!")
print(f"📁 Рабочая директория: {project_root}")
print(f"📦 Доступные стратегии: GridSearchStrategy, RandomSearchStrategy")
print(f"📊 Доступные хранилища: InMemoryResultsStore")
print(f"📈 Доступные репортеры: OptimizationReporter")


In [75]:
# Ячейка 1: Импорты и настройка
import sys
sys.path.append('src')
from data import CSVLoader
import pandas as pd
import numpy as np

from lightweight_charts import JupyterChart
# Ячейка 2: Настройка данных
DATA_DIR = r"G:\My Drive\data_fut"
TICKER = "IMOEXF"
loader = CSVLoader(DATA_DIR, use_cache=True)

# Ячейка 3: Загрузка данных
start_date = '2025-01-15'
end_date = '2025-10-15'

timeframe = 10

df = loader.load(TICKER, timeframe=timeframe, start_date=start_date, end_date=end_date)
print(f"Данные: {len(df)} баров, период: {df.index[0]} - {df.index[-1]}")

# Ячейка 4: Ваш анализ (например, Volatility Median)
# Здесь можете использовать функции из VM_base_optimize.py

Данные: 17903 баров, период: 2025-01-15 10:00:00 - 2025-10-14 23:50:00


In [76]:
display(df['CLOSE'].agg(
    min_price='min',
    max_price='max',
    avg_price='mean',
    median_price='median',
    std_price='std',
    var_price='var',
    n='count'
))


min_price        2528.500000
max_price        3381.500000
avg_price        2895.663269
median_price     2867.500000
std_price         176.716414
var_price       31228.691083
n               17903.000000
Name: CLOSE, dtype: float64

In [77]:
# Простая функция вместо класса SignalFactory
def create_signal_config(parameters):
    """Разделяем параметры по назначению"""
    return SignalConfig(
        signal=SignalRegistry.get("SlopeSignal"),
        signal_params={'threshold': 0.001},  # SlopeSignal: threshold, absolute
        indicator_params={
            'KATR': parameters.get('katr'),
            'PerATR': parameters.get('peratr'),
            'SMA': parameters.get('sma'),
            'MinRA': parameters.get('minra'),
            'FlATR': parameters.get('flatr'),
            'FlHL': parameters.get('flhl')
        },
        backtest_params={
            # entry_price_type/n_contracts можно переопределить здесь при необходимости
            # 'entry_price_type': 'close',
            # 'n_contracts': 1,
        }
    )

# Простая функция вместо класса IndicatorBuilder
def build_indicator(df, params, *, verbose=False):
    """Создаем индикатор с параметрами"""
    vm_ind = IndicatorRegistry.get("VolatilityMedian")

    # Параметры для VolatilityMedian: KATR, PerATR, SMA, MinRA, FlATR, FlHL
    param_mapping = {
        'katr': 'KATR',
        'peratr': 'PerATR',
        'sma': 'SMA',
        'minra': 'MinRA',
        'flatr': 'FlATR',
        'flhl': 'FlHL'
    }

    indicator_params = {}
    for param_key, param_value in (params or {}).items():
        normalized_key = str(param_key).lower()
        mapped_key = param_mapping.get(normalized_key)
        if mapped_key and param_value is not None:
            indicator_params[mapped_key] = param_value

    if verbose:
        print(f"Параметры для индикатора: {indicator_params}")

    return vm_ind.calculate(df, **indicator_params)


In [78]:
# Простой способ - создание из ваших данных
your_params = {
    "KATR": [4, 5, 6, 7, 8, 9, 10, 11],
    "PerATR": [2, 10, 30, 80], 
    "SMA": [1, 2, 3, 4, 10],
    "MinRA": [0],
    "FlATR": [0],
    "FlHL": [0]
}

# Преобразование в ParameterSpace
definitions = []
for name, values in your_params.items():
    definitions.append(ParameterDefinition(
        name=name.lower(),  # приводим к нижнему регистру
        values=values
    ))

parameter_space = ParameterSpace.from_definitions(definitions)

In [79]:
# Создаем оптимизатор
executor = OptimizationExecutor(
    backtest_engine=BacktestEngine(),
    parameter_space=parameter_space,
    strategy=GridSearchStrategy(parameter_space),
    signal_factory=create_signal_config,  # ← Простая функция!
    results_store=InMemoryResultsStore(),
    indicator_builder=build_indicator,    # ← Простая функция!
    # ↓↓↓ новые настройки производительности/стабильности ↓↓↓
    checkpoint_interval=1000,              # было 1 → станет реже и быстрее
    per_candidate_timeout=None,           # убрать пер-кандидатные пулы
    default_max_workers=1,                # дефолт: одиночный режим
    checkpoint_enabled=True               # оставить включённым (но с редким интервалом)
    
)

print("✅ Оптимизатор создан!")

✅ Оптимизатор создан!


In [80]:
# result_indices = [88]
# verbose = True

from itertools import cycle

_BOKEH_OUTPUT_NOTEBOOK_LOADED = globals().get('_BOKEH_OUTPUT_NOTEBOOK_LOADED', False)

def _ensure_bokeh_inline():
    """Загружает BokehJS inline, чтобы графики отображались без доступа к CDN."""
    global _BOKEH_OUTPUT_NOTEBOOK_LOADED
    if not _BOKEH_OUTPUT_NOTEBOOK_LOADED:
        from bokeh.io import output_notebook
        from bokeh.resources import INLINE
        output_notebook(resources=INLINE, verbose=False, hide_banner=True)
        _BOKEH_OUTPUT_NOTEBOOK_LOADED = True

def _collect_indicator_params(parameters):
    """Преобразует параметры результата в формат индикатора VolatilityMedian."""
    mapping = {
        'katr': 'KATR',
        'peratr': 'PerATR',
        'sma': 'SMA',
        'minra': 'MinRA',
        'flatr': 'FlATR',
        'flhl': 'FlHL',
    }
    extracted = {}
    for raw_key, value in (parameters or {}).items():
        key = mapping.get(str(raw_key).lower())
        if key and value is not None:
            extracted[key] = value
    return extracted

def _ensure_series(signals, index):
    """Возвращает Series с сигналами, выровненными по индексу цен."""
    import pandas as pd
    if hasattr(signals, 'index'):
        series = signals.reindex(index)
        return series.ffill().fillna(0)
    return pd.Series(signals, index=index).fillna(0)

def _run_backtest_for_result(result_index, *, verbose=True):
    """Пересчитывает бэктест для выбранного результата и возвращает данные."""
    if not 0 <= result_index < len(results):
        if verbose:
            print(f"Ошибка: индекс {result_index} вне диапазона доступных результатов (0..{len(results) - 1})")
        return None

    target_result = results[result_index]
    signal_config = create_signal_config(target_result.parameters)

    indicator_series = getattr(signal_config, 'indicator', None)
    if indicator_series is None:
        indicator_series = build_indicator(df, target_result.parameters, verbose=verbose)
    if indicator_series is not None and hasattr(indicator_series, 'reindex'):
        indicator_series = indicator_series.reindex(df.index).ffill()

    raw_signals = signal_config.signal.generate(df, indicator_series, **signal_config.signal_params)
    signals = _ensure_series(raw_signals, df.index)

    backtest_engine = BacktestEngine()
    backtest_params = dict(getattr(signal_config, 'backtest_params', {}))
    backtest_result = backtest_engine.run(df, signals, **backtest_params)

    return {
        'result_index': result_index,
        'target_result': target_result,
        'indicator_params': _collect_indicator_params(target_result.parameters),
        'signals': signals,
        'backtest_result': backtest_result,
        'equity': backtest_result.get('equity'),
    }

def build_equity_for_result(result_index, *, verbose=True):
    """Строит эквити для отдельного результата и выводит ключевые метрики."""
    run_data = _run_backtest_for_result(result_index, verbose=verbose)
    if not run_data:
        return None

    target_result = run_data['target_result']
    backtest_result = run_data['backtest_result']
    equity = run_data['equity']

    print(f"Строим эквити для результата #{result_index}")
    print(
        "Метрики из оптимизации: Total Return={:.4f}, Max DD={:.4f}".format(
            target_result.metrics.get('total_return', 0),
            target_result.metrics.get('max_drawdown', 0),
        )
    )

    if equity is not None:
        print(f"Эквити построена: {len(equity)} точек")
        print(
            "Повторный бэктест: Total Return={:.4f}, Max DD={:.4f}".format(
                backtest_result.get('total_return', 0),
                backtest_result.get('max_drawdown', 0),
            )
        )
        return equity

    print('Ошибка: не удалось получить эквити из результата бэктеста')
    return None

def plot_equities_for_results(result_indices, *, verbose=True):
    """Строит один график с эквити и таблицу метрик для выбранных индексов."""
    if not result_indices:
        print('Список индексов пуст. Укажите хотя бы один индекс результата.')
        return None

    prepared_runs = []
    for idx in result_indices:
        run_data = _run_backtest_for_result(idx, verbose=verbose)
        if run_data and run_data.get('equity') is not None:
            prepared_runs.append(run_data)

    if not prepared_runs:
        print('Не удалось построить эквити ни для одного из выбранных индексов.')
        return None

    _ensure_bokeh_inline()

    from bokeh.layouts import column
    from bokeh.models import ColumnDataSource, DataTable, NumberFormatter, TableColumn, HoverTool
    from bokeh.palettes import Category10, Category20, Turbo256
    from bokeh.plotting import figure
    from bokeh.io import show

    n = len(prepared_runs)
    if n <= 10:
        palette = list(Category10[10][:n])
    elif n <= 20:
        palette = list(Category20[20][:n])
    else:
        step = max(len(Turbo256) // n, 1)
        palette = [Turbo256[(i * step) % len(Turbo256)] for i in range(n)]
    color_cycle = cycle(palette)

    p = figure(
        title=f"Equity Curve (индексы: {result_indices})",
        x_axis_type='datetime',
        width=900,
        height=420,
        tools='pan,wheel_zoom,box_zoom,reset,save'
    )
    p.add_tools(HoverTool(tooltips=[('Время', '@x{%F %T}'), ('Эквити', '@y{0.00}')], formatters={'@x': 'datetime'}))

    table_rows = {
        'result_index': [],
        'indicator': [],
        'total_return': [],
        'max_drawdown': [],
        'n_trades': [],
        'wave_ratio': [],
    }

    for run_data, color in zip(prepared_runs, color_cycle):
        equity = run_data['equity']
        metrics = run_data['backtest_result']
        label = f"Equity #{run_data['result_index']}"
        p.line(equity.index, equity.values, line_width=2, color=color, legend_label=label)

        table_rows['result_index'].append(run_data['result_index'])
        indicator_description = ', '.join(f"{k}={v}" for k, v in run_data['indicator_params'].items()) or '—'
        table_rows['indicator'].append(indicator_description)
        table_rows['total_return'].append(metrics.get('total_return', float('nan')))
        table_rows['max_drawdown'].append(metrics.get('max_drawdown', float('nan')))
        table_rows['n_trades'].append(metrics.get('n_trades', float('nan')))
        wave_ratio = metrics.get('mean_wave_mfe_ratio')
        if wave_ratio is None:
            wave_ratio = metrics.get('wave_ratio')
        table_rows['wave_ratio'].append(wave_ratio if wave_ratio is not None else float('nan'))

    p.legend.location = 'top_left'
    p.legend.click_policy = 'hide'
    p.grid.grid_line_alpha = 0.25
    p.xaxis.axis_label = 'Время'
    p.yaxis.axis_label = 'Эквити'

    source = ColumnDataSource(table_rows)
    columns = [
        TableColumn(field='result_index', title='Индекс результата'),
        TableColumn(field='indicator', title='Параметры индикатора'),
        TableColumn(field='total_return', title='Total Return', formatter=NumberFormatter(format='0.0000')),
        TableColumn(field='max_drawdown', title='Max DD', formatter=NumberFormatter(format='0.0000')),
        TableColumn(field='n_trades', title='N trades', formatter=NumberFormatter(format='0')),
        TableColumn(field='wave_ratio', title='Wave ratio', formatter=NumberFormatter(format='0.0000')),
    ]

    data_table = DataTable(
        source=source,
        columns=columns,
        width=900,
        height=220,
        index_position=None,
        autosize_mode='fit_columns'
    )

    show(column(p, data_table))

    print(f"Эквити построены для индексов: {[run['result_index'] for run in prepared_runs]}")
    return prepared_runs


# runs_summary = plot_equities_for_results(result_indices, verbose=verbose)


In [81]:
# === РАСЧЕТЫ ПЕРЕД ЗАПУСКОМ ===
print("📊 АНАЛИЗ ПАРАМЕТРОВ ОПТИМИЗАЦИИ")
print("=" * 50)

# 1. Размер данных
print(f"📈 Будет протестировано: {len(df):,} свечей")
print(f"📅 Период данных: {df.index[0].strftime('%Y-%m-%d')} - {df.index[-1].strftime('%Y-%m-%d')}")

# 2. Расчет количества комбинаций
total_combinations = 1
param_details = []

for name, values in your_params.items():
    count = len(values)
    total_combinations *= count
    param_details.append(f"  {name}: {count} значений {values}")

print(f"\n🔢 ПАРАМЕТРЫ ОПТИМИЗАЦИИ:")
for detail in param_details:
    print(detail)

print(f"\n🧮 РАСЧЕТ КОМБИНАЦИЙ:")
print(f"  Всего параметров: {len(your_params)}")
print(f"  Комбинаций: {total_combinations:,}")
print(f"  Формула: {' × '.join([str(len(values)) for values in your_params.values()])} = {total_combinations:,}")

# 3. Оценка времени выполнения
estimated_time_per_combination = 0.9  # секунд на комбинацию (примерно)
total_time_seconds = total_combinations * estimated_time_per_combination
total_time_minutes = total_time_seconds / 60
total_time_hours = total_time_minutes / 60

print(f"\n⏱️ ОЦЕНКА ВРЕМЕНИ:")
print(f"  ~{estimated_time_per_combination} сек на комбинацию")
print(f"  Общее время: {total_time_seconds:.1f} сек ({total_time_minutes:.1f} мин)")
if total_time_hours >= 1:
    print(f"  Или: {total_time_hours:.1f} часов")

# 4. Предупреждения
if total_combinations > 10000:
    print(f"\n⚠️ ВНИМАНИЕ: Большое количество комбинаций!")
    print(f"   Рекомендуется уменьшить параметры или использовать RandomSearchStrategy")
    
if total_combinations > 100000:
    print(f"\n🚨 КРИТИЧНО: Очень много комбинаций!")
    print(f"   Рассмотрите возможность:")
    print(f"   - Уменьшения количества значений параметров")
    print(f"   - Использования RandomSearchStrategy с max_iterations=1000")
    print(f"   - Разделения оптимизации на этапы")

print("\n" + "=" * 50)



📊 АНАЛИЗ ПАРАМЕТРОВ ОПТИМИЗАЦИИ
📈 Будет протестировано: 17,903 свечей
📅 Период данных: 2025-01-15 - 2025-10-14

🔢 ПАРАМЕТРЫ ОПТИМИЗАЦИИ:
  KATR: 8 значений [4, 5, 6, 7, 8, 9, 10, 11]
  PerATR: 4 значений [2, 10, 30, 80]
  SMA: 5 значений [1, 2, 3, 4, 10]
  MinRA: 1 значений [0]
  FlATR: 1 значений [0]
  FlHL: 1 значений [0]

🧮 РАСЧЕТ КОМБИНАЦИЙ:
  Всего параметров: 6
  Комбинаций: 160
  Формула: 8 × 4 × 5 × 1 × 1 × 1 = 160

⏱️ ОЦЕНКА ВРЕМЕНИ:
  ~0.9 сек на комбинацию
  Общее время: 144.0 сек (2.4 мин)



In [82]:
import time
# === ЗАПУСКАЕМ ОПТИМИЗАЦИЮ ===
print("🚀 Начинаем оптимизацию...")
start_time = time.time()

run_dir = executor.run(df)  # фиксируем одиночный воркер

end_time = time.time()
actual_time = end_time - start_time

# === РЕЗУЛЬТАТЫ ===
results = list(executor.results_store)
print(f"✅ Завершено! Получено {len(results)} результатов")
print(f"⏱️ Фактическое время: {actual_time:.1f} сек ({actual_time/60:.1f} мин)")
print(f"📊 Скорость: {len(results)/actual_time:.1f} комбинаций/сек")
best_result = max(results, key=lambda r: r.metrics.get('total_return', 0))
print(f"🏆 Лучший результат: {best_result.parameters}")
print(f"📊 Метрики: {best_result.metrics}")
print(f"📁 Директория запуска: {run_dir}")

🚀 Начинаем оптимизацию...
✅ Завершено! Получено 160 результатов
⏱️ Фактическое время: 179.0 сек (3.0 мин)
📊 Скорость: 0.9 комбинаций/сек
🏆 Лучший результат: {'katr': 6, 'peratr': 2, 'sma': 3, 'minra': 0, 'flatr': 0, 'flhl': 0}
📊 Метрики: {'total_return': 0.009736797399999775, 'max_drawdown': 0.003444425959847508, 'recovery_factor': 2.826827318544203, 'sharpe': 0.18234569666550107, 'sortino': 0.23713638588819438, 'n_flips': 39, 'n_trades': 40, 'win_rate': 0.5, 'profit_factor': 2.2842735551414135, 'waves_count': 39, 'mean_wave_mfe': 107.1025641025641, 'mean_wave_mfe_ratio': 4.101700020174295}
📁 Директория запуска: output\optimization\20251027_110914


In [83]:
# === ТАБЛИЦА РЕЗУЛЬТАТОВ ТЕСТИРОВАНИЯ ===
import pandas as pd

# Создаем DataFrame с результатами
results_data = []
for i, result in enumerate(results):
    row = {
        'Индекс': i,
        'KATR': result.parameters.get('katr'),
        'PerATR': result.parameters.get('peratr'), 
        'SMA': result.parameters.get('sma'),
        'Total Return': result.metrics.get('total_return', 0),
        'Max Drawdown': result.metrics.get('max_drawdown', 0),
        'Recovery Factor': result.metrics.get('recovery_factor', 0),
        'Sharpe': result.metrics.get('sharpe', 0),
        'Sortino': result.metrics.get('sortino', 0),
        'Win Rate': result.metrics.get('win_rate', 0),
        'Profit Factor': result.metrics.get('profit_factor', 0),
        'N Trades': result.metrics.get('n_trades', 0),
        'Wave_ratio': result.metrics.get('mean_wave_mfe_ratio', 0)
    }
    results_data.append(row)

# Создаем DataFrame и сортируем по Total Return
results_df = pd.DataFrame(results_data)
results_df = results_df.sort_values('Sharpe', ascending=False).reset_index(drop=True)

print("📊 ТАБЛИЦА РЕЗУЛЬТАТОВ ОПТИМИЗАЦИИ")
print("=" * 80)
print(f"Всего результатов: {len(results_df)}")
print(f"Лучший результат (индекс {results_df.index[0]}):")
print(f"  Total Return: {results_df.iloc[0]['Total Return']:.4f}")
print(f"  Max Drawdown: {results_df.iloc[0]['Max Drawdown']:.4f}")
print(f"  Sharpe: {results_df.iloc[0]['Sharpe']:.4f}")
print("=" * 80)

# Отображаем топ-10 результатов
print("\n🏆 ТОП-10 РЕЗУЛЬТАТОВ:")
display(results_df.head(10).round(4))


📊 ТАБЛИЦА РЕЗУЛЬТАТОВ ОПТИМИЗАЦИИ
Всего результатов: 160
Лучший результат (индекс 0):
  Total Return: 0.0097
  Max Drawdown: 0.0034
  Sharpe: 0.1823

🏆 ТОП-10 РЕЗУЛЬТАТОВ:


,Индекс,KATR,PerATR,SMA,Total Return,Max Drawdown,Recovery Factor,Sharpe,Sortino,Win Rate,Profit Factor,N Trades,Wave_ratio
0,42,6,2,3,0.0097,0.0034,2.8268,0.1823,0.2371,0.5000,2.2843,40,4.1017
1,36,5,80,2,0.0091,0.0046,1.9649,0.1706,0.2190,0.4545,1.8272,44,6.3742
2,100,9,2,1,0.0090,0.0045,1.9998,0.1689,0.2186,0.4773,1.9172,44,2.8031
3,61,7,2,2,0.0088,0.0034,2.5747,0.1646,0.2103,0.4792,1.9324,48,4.9150
4,37,5,80,3,0.0086,0.0055,1.5701,0.1617,0.2054,0.4706,1.9289,34,4.9609
5,55,6,80,1,0.0086,0.0039,2.2092,0.1616,0.2071,0.4286,1.9212,42,4.6119
6,145,11,10,1,0.0084,0.0031,2.7246,0.1570,0.2029,0.5417,2.6615,24,6.2472
7,57,6,80,3,0.0081,0.0044,1.8440,0.1527,0.1935,0.4286,1.9442,28,2.2892
8,146,11,10,2,0.0081,0.0031,2.6300,0.1515,0.1953,0.6000,2.5597,20,2.2138
9,127,10,10,3,0.0077,0.0047,1.6453,0.1438,0.1848,0.5556,2.5877,18,4.1434


In [86]:
selected = [42, 127]
plot_equity_curves(selected, title="Top-3 strategies", verbose=False)

Строим эквити для результата #42
Метрики из оптимизации: Total Return=0.0097, Max DD=0.0034
Эквити построена: 17903 точек
Повторный бэктест: Total Return=0.0097, Max DD=0.0034
Строим эквити для результата #127
Метрики из оптимизации: Total Return=0.0077, Max DD=0.0047
Эквити построена: 17903 точек
Повторный бэктест: Total Return=0.0077, Max DD=0.0047


,Начальная эквити,Конечная эквити,Доходность %,Макс. просадка %
Индекс,,,,
42,100000.0,100973.68,0.97,0.34
127,100000.0,100765.98,0.77,0.47


,Начальная эквити,Конечная эквити,Доходность %,Макс. просадка %
Индекс,,,,
42,100000.0,100973.67974,0.973680,0.344443
127,100000.0,100765.98085,0.765981,0.465548


In [68]:
result_index = 88
# === ПЕРЕЗАГРУЗКА ФУНКЦИЙ (ВАЖНО!) ===
# Перезагружаем исправленную функцию build_indicator
def build_indicator(df, params, *, verbose=True):
    """Создаем индикатор с параметрами (ИСПРАВЛЕННАЯ ВЕРСИЯ)"""
    vm_ind = IndicatorRegistry.get("VolatilityMedian")

    # Фильтруем только нужные параметры для VolatilityMedian
    # Параметры для VolatilityMedian: KATR, PerATR, SMA, MinRA, FlATR, FlHL
    param_mapping = {
        'katr': 'KATR',
        'peratr': 'PerATR',
        'sma': 'SMA',
        'minra': 'MinRA',
        'flatr': 'FlATR',
        'flhl': 'FlHL'
    }

    indicator_params = {}
    for param_key, param_value in (params or {}).items():
        normalized_key = str(param_key).lower()
        mapped_key = param_mapping.get(normalized_key)
        if mapped_key and param_value is not None:
            indicator_params[mapped_key] = param_value

    if verbose:
        print(f"🔧 Параметры для индикатора: {indicator_params}")
    return vm_ind.calculate(df, **indicator_params)

print("✅ Функция build_indicator перезагружена с исправлениями!")


def _ensure_series(signals, index):
    """Приводим сигналы к Series с корректным индексом"""
    if hasattr(signals, "index"):
        return signals
    import pandas as pd
    return pd.Series(signals, index=index)


# === ПОСТРОЕНИЕ ЭКВИТИ ПО ИНДЕКСУ РЕЗУЛЬТАТА ===
def build_equity_for_result(result_index, *, verbose=True):
    """Строит эквити для результата по индексу"""
    if not 0 <= result_index < len(results):
        if verbose:
            print(f"❌ Ошибка: Индекс {result_index} выходит за пределы доступных результатов (0..{len(results) - 1})")
        return None

    # Получаем результат по индексу
    target_result = results[result_index]
    if verbose:
        print(f"📊 Строим эквити для результата #{result_index}")
        print(
            "Метрики: Total Return={:.4f}, Max DD={:.4f}".format(
                target_result.metrics.get('total_return', 0),
                target_result.metrics.get('max_drawdown', 0)
            )
        )

    # Создаем конфигурацию сигнала для этого результата
    signal_config = create_signal_config(target_result.parameters)

    # Строим индикатор
    indicator_data = build_indicator(df, target_result.parameters, verbose=verbose)
    indicator_series = getattr(signal_config, 'indicator', None) or indicator_data
    if indicator_series is not None and hasattr(indicator_series, "reindex"):
        indicator_series = indicator_series.reindex(df.index).ffill()

    # Генерируем сигналы на основе конфигурации
    raw_signals = signal_config.signal.generate(df, indicator_series, **signal_config.signal_params)
    signals = _ensure_series(raw_signals, df.index)

    # Создаем бэктест движок и применяем параметры из конфигурации
    backtest_engine = BacktestEngine()
    backtest_params = dict(getattr(signal_config, 'backtest_params', {}))
    backtest_result = backtest_engine.run(df, signals, **backtest_params)

    # Извлекаем эквити
    equity = backtest_result.get('equity')
    if equity is not None:
        if verbose:
            print(f"✅ Эквити построена: {len(equity)} точек")
            print(
                "Повторный бэктест: Total Return={:.4f}, Max DD={:.4f}".format(
                    backtest_result.get('total_return', 0),
                    backtest_result.get('max_drawdown', 0)
                )
            )
        return equity

    if verbose:
        print("❌ Ошибка: Не удалось получить эквити из результата бэктеста")
    return None


def plot_equity_curves(strategy_indices, *, title=None, palette=None, width=900, height=420, verbose=False):
    """Отображает эквити для списка индексов стратегий на одном графике."""
    import numpy as np
    import pandas as pd
    from bokeh.plotting import figure, show, output_notebook
    from bokeh.models import HoverTool, ColumnDataSource
    from bokeh.palettes import Category10, Turbo256
    from IPython.display import display

    if isinstance(strategy_indices, pd.Series):
        indices = strategy_indices.tolist()
    elif hasattr(strategy_indices, 'tolist') and not isinstance(strategy_indices, (list, tuple)):
        try:
            indices = list(strategy_indices)
        except TypeError:
            indices = [strategy_indices]
    elif isinstance(strategy_indices, (list, tuple, set)):
        indices = list(strategy_indices)
    else:
        indices = [strategy_indices]

    unique_indices = []
    for raw_idx in indices:
        try:
            idx = int(raw_idx)
        except (TypeError, ValueError):
            if verbose:
                print(f"⚠️ Пропускаем некорректный индекс: {raw_idx!r}")
            continue
        if idx not in unique_indices:
            unique_indices.append(idx)

    if not unique_indices:
        if verbose:
            print("⚠️ Нет допустимых индексов стратегий для отображения.")
        return None

    curves = []
    stats_rows = []
    for idx in unique_indices:
        equity_series = build_equity_for_result(idx, verbose=verbose)
        if equity_series is None or (hasattr(equity_series, 'empty') and equity_series.empty):
            if verbose:
                print(f"⚠️ Эквити для стратегии #{idx} не построена.")
            continue
        series = equity_series.sort_index()
        curves.append((idx, series))

        start_value = float(series.iloc[0])
        end_value = float(series.iloc[-1])
        cumulative_max = series.cummax()
        drawdown = ((cumulative_max - series) / cumulative_max).replace([np.inf, -np.inf], 0.0).fillna(0.0)
        max_drawdown = float(drawdown.max() * 100)

        stats_rows.append({
            'Индекс': idx,
            'Начальная эквити': start_value,
            'Конечная эквити': end_value,
            'Доходность %': (end_value / start_value - 1) * 100 if start_value else np.nan,
            'Макс. просадка %': max_drawdown,
        })

    if not curves:
        if verbose:
            print("⚠️ Не удалось построить эквити ни для одной стратегии.")
        return None

    def _resolve_palette(n, custom_palette):
        if custom_palette:
            palette_list = list(custom_palette)
            if len(palette_list) < n:
                repeats = (n + len(palette_list) - 1) // len(palette_list)
                palette_list = (palette_list * repeats)[:n]
            return palette_list
        if n <= 10:
            return Category10[10][:n]
        step = max(len(Turbo256) // n, 1)
        return [Turbo256[i * step] for i in range(n)]

    colors = _resolve_palette(len(curves), palette)

    output_notebook(hide_banner=True)

    plot_title = title or "Эквити выбранных стратегий"
    p = figure(
        title=plot_title,
        x_axis_type='datetime',
        width=width,
        height=height,
        tools='pan,wheel_zoom,box_zoom,reset,save'
    )

    hover = HoverTool(
        tooltips=[
            ("Стратегия", "@strategy"),
            ("Время", "@x{%F %T}"),
            ("Эквити", "@y{0.00}")
        ],
        formatters={"@x": "datetime"},
        mode="vline"
    )
    p.add_tools(hover)

    for color, (idx, series) in zip(colors, curves):
        source = ColumnDataSource({
            'x': series.index,
            'y': series.values,
            'strategy': [f'#{idx}'] * len(series),
        })
        p.line('x', 'y', source=source, line_width=2, line_color=color, legend_label=f'#{idx}')

    p.legend.location = 'top_left'
    p.legend.click_policy = 'hide'
    p.grid.grid_line_alpha = 0.25
    p.xaxis.axis_label = 'Время'
    p.yaxis.axis_label = 'Эквити'

    show(p)

    if stats_rows:
        stats_df = pd.DataFrame(stats_rows).set_index('Индекс')
        display(stats_df.round(2))
        return stats_df

    return p


# === ИНТЕРАКТИВНОЕ ПОСТРОЕНИЕ ЭКВИТИ ===
# Введите индекс результата для построения эквити (0-83)
# result_index = 54  # ← ИЗМЕНИТЕ ЭТО ЗНАЧЕНИЕ НА НУЖНЫЙ ИНДЕКС
verbose = True

equity = build_equity_for_result(result_index, verbose=verbose)

if equity is not None:
    print(f"📈 Эквити готова для отображения: {len(equity)} точек")
    print(f"Начальное значение: {equity.iloc[0]:.2f}")
    print(f"Конечное значение: {equity.iloc[-1]:.2f}")
    print(f"Максимум: {equity.max():.2f}")
    print(f"Минимум: {equity.min():.2f}")
else:
    print("❌ Не удалось построить эквити")

# Пример: plot_equity_curves(results_df.head(3)['Индекс'])


✅ Функция build_indicator перезагружена с исправлениями!
📊 Строим эквити для результата #88
Метрики: Total Return=0.0077, Max DD=0.0062
🔧 Параметры для индикатора: {'KATR': 10, 'PerATR': 10, 'SMA': 4, 'MinRA': 0, 'FlATR': 0, 'FlHL': 0}
✅ Эквити построена: 35569 точек
Повторный бэктест: Total Return=0.0077, Max DD=0.0062
📈 Эквити готова для отображения: 35569 точек
Начальное значение: 100000.00
Конечное значение: 100767.34
Максимум: 100896.47
Минимум: 99773.10


In [ ]:
# === ОТОБРАЖЕНИЕ ЭКВИТИ С BOKEH ===
if equity is not None:
    from bokeh.plotting import figure, show, output_notebook
    from bokeh.models import HoverTool
    
    output_notebook()
    
    # Создаем график
    p = figure(
        title=f'Equity Curve (Результат #{result_index})', 
        x_axis_type='datetime', 
        width=900, 
        height=420, 
        tools='pan,wheel_zoom,box_zoom,reset,save'
    )
    
    # Добавляем линию эквити
    p.line(equity.index, equity.values, line_width=2, color='#2962FF', legend_label='Equity')
    
    # Настраиваем hover
    p.add_tools(HoverTool(
        tooltips=[
            ('Время', '@x{%F %T}'), 
            ('Эквити', '@y{0.00}')
        ], 
        formatters={'@x': 'datetime'}
    ))
    
    # Настройки графика
    p.legend.location = 'top_left'
    p.grid.grid_line_alpha = 0.25
    p.xaxis.axis_label = 'Время'
    p.yaxis.axis_label = 'Эквити'
    
    # Показываем график
    show(p)
    
    # Дополнительная статистика
    print(f"\n📊 СТАТИСТИКА ЭКВИТИ:")
    print(f"  Начальная эквити: {equity.iloc[0]:.2f}")
    print(f"  Конечная эквити: {equity.iloc[-1]:.2f}")
    print(f"  Общая доходность: {(equity.iloc[-1] / equity.iloc[0] - 1) * 100:.2f}%")
    print(f"  Максимальная эквити: {equity.max():.2f}")
    print(f"  Минимальная эквити: {equity.min():.2f}")
    print(f"  Максимальная просадка: {((equity.cummax() - equity) / equity.cummax()).max() * 100:.2f}%")
else:
    print("❌ Эквити не построена. Сначала выполните предыдущую ячейку.")
